# Ngày 4: PAIR RDD

### 1. Pair RDDs

a. Khái niệm:
- Khái niệm: Pair RDDs là các RDD chứa các cặp dữ liệu dạng (key, value), giúp dễ dàng thực hiện các phép toán dựa trên khóa.
- Ví dụ: Từ một danh sách các giao dịch bán hàng, tạo Pair RDD với mã sản phẩm là "key" và số lượng bán là "value". ('MaSP', soluong)
- Ở các bài tập các buổi trước, việc đếm word count hay tính tổng doanh thu dựa theo Category các bạn đều đã tạo qua Pair RDDS để làm bài.

b. Cách tạo Pair RDDS:
- <p style = "color: blue">Tạo từ Map và FlatMap: </p>

```python
        rdd = sc.textFile("path")
        # Dem word count trong van ban
        word_count = rdd.flatMap(lambda x: x.split()) \
                        .map(lambda word : (word,1)) \
                        .reduceByKey(lambda x, y: x+y)
```

- <p style = "color: blue">Tạo từ một một danh sách các cặp (key, value)</p>

```python
        sales_data = [("ProductA", 30), ("ProductB", 20), ("ProductA", 15)]
        pair_rdd = sc.parallelize(sales_data)
        # Output: [('ProductA', 30), ('ProductB', 20), ('ProductA', 15)]
```

- <p style = "color: blue">Sử dụng hàm KeyBy </p>

```python
        products_rdd = sc.parallelize([("ProductA", 100, "CategoryX"),
                                         ("ProductB", 200, "CategoryY")])
                                         
        pair_rdd = products_rdd.keyBy(lambda x: x[0])  # Khóa là tên sản phẩm
        # Output: [('ProductA', ('ProductA', 100, 'CategoryX')), 
        #               ('ProductB', ('ProductB', 200, 'CategoryY'))]

```

### 2. Các phép toán trên Pair RDDs



**a. ReduceByKey**: Gộp các giá trị dựa trên khóa bằng một hàm tổng hợp (như cộng, trừ)\


**b. groupByKey**: Gom các giá trị cùng khóa vào một danh sách
```python
        # Ví dụ: Gom tất cả các giao dịch vào một danh sách
        sales_rdd = sc.parallelize([("ProductA", 30), ("ProductB", 20), ("ProductA", 15)])
        grouped_sales = sales_rdd.groupByKey().mapValues(list)
        # Output: [('ProductA', [30, 15]), ('ProductB', [20])]

```

**c. sortByKey**: Sắp xếp Pair RDDs dựa trên khóa
```python
        products_rdd = sc.parallelize([("ProductB", 200), ("ProductA", 100), ("ProductC", 150)])
        sorted_products = products_rdd.sortByKey()
        # Output: [('ProductA', 100), ('ProductB', 200), ('ProductC', 150)]


```


**d. join**: Ghép hai Pair RDDs dựa trên khóa chung
```python
        sales_rdd = sc.parallelize([("ProductA", 30), ("ProductB", 20), ("ProductA", 15)])

        product_info_rdd = sc.parallelize([("ProductA", "Description of A"), 
                                        ("ProductB", "Description of B")])

        joined_rdd = sales_rdd.join(product_info_rdd)
        # Output: [('ProductA', (30, 'Description of A')), 
        # ('ProductA', (15, 'Description of A')), ('ProductB', (20, 'Description of B'))]

```

**e. cogroup**: Ghép hai Pair RDD có cùng khóa nhưng giá trị khác nhau thành các tập giá trị.

Ví dụ: Kết hợp điểm của sinh viên từ hai môn học để xem tất cả điểm theo từng sinh viên.
```python
        subject1_rdd = sc.parallelize([("Student1", 85), ("Student2", 78), ("Student1", 90)])
        subject2_rdd = sc.parallelize([("Student1", 88), ("Student2", 92), ("Student3", 75)])
        combined_scores = subject1_rdd.cogroup(subject2_rdd).mapValues(lambda x: (list(x[0]), list(x[1])))
        # Output: [('Student1', ([85, 90], [88])), ('Student2', ([78], [92])), ('Student3', ([], [75]))]

```


## Bài tập

**Bài 1: Tìm sản phẩm bán chạy nhất trong mỗi tháng từ một RDD chứa dữ liệu bán hàng.**

Ý tưởng: Tạo ra các rdd = [((Thang, SanPham), SoLuong), .........]\
Sau đó lại tạo ra rdd = [((thang),(SanPham, SoLuong)), ..............] rồi dùng groupByKey để group theo thang\
và sau đó lọc ra theo SoLuong lon nhat

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Pair RDD") \
    .getOrCreate()

24/11/07 22:34:38 WARN Utils: Your hostname, ubunchuu-Test resolves to a loopback address: 127.0.1.1; using 10.0.224.45 instead (on interface wlo1)
24/11/07 22:34:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/07 22:34:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext


In [4]:
# Load to memory
sales_rdd = sc.textFile("material_1/online_sales_data.csv")

In [5]:
sales_rdd.take(5)

['10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card',
 '10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal',
 "10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card",
 '10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card',
 '10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal']

In [6]:
# Hàm kiểm tra và xử lý lỗi
# Tao ra cac pair rdd là ((thang,sanpham), soluong)
def parse_line(line):
    fields = line.split(",")
    # Kiểm tra xem dòng có đủ cột và dữ liệu có hợp lệ không
    if len(fields) == 9:  # Số cột hợp lệ trong file
        try:
            # Chuyển đổi số lượng từ chuỗi thành số nguyên
            so_luong = int(fields[4])
            thang = fields[1][:7]  # Lấy tháng từ cột ngày
            san_pham = fields[2]    # Lấy tên loai san pham
            return (thang, san_pham), so_luong
        except ValueError:
            return None  # Trả về None nếu không thể chuyển đổi số lượng
    else:
        return None  # Trả về None nếu thiếu trường

In [7]:
pair_rdd = sales_rdd.map(parse_line).filter(lambda x: x is not None)

In [8]:
pair_rdd.collect()

[(('2024-01', 'Electronics'), 2),
 (('2024-01', 'Home Appliances'), 1),
 (('2024-01', 'Clothing'), 3),
 (('2024-01', 'Books'), 4),
 (('2024-01', 'Beauty Products'), 1),
 (('2024-01', 'Sports'), 5),
 (('2024-01', 'Electronics'), 1),
 (('2024-01', 'Home Appliances'), 2),
 (('2024-01', 'Clothing'), 6),
 (('2024-01', 'Books'), 2),
 (('2024-01', 'Beauty Products'), 1),
 (('2024-01', 'Sports'), 3),
 (('2024-01', 'Electronics'), 2),
 (('2024-01', 'Home Appliances'), 1),
 (('2024-01', 'Clothing'), 2),
 (('2024-01', 'Beauty Products'), 1),
 (('2024-01', 'Sports'), 4),
 (('2024-01', 'Electronics'), 2),
 (('2024-01', 'Home Appliances'), 1),
 (('2024-01', 'Clothing'), 3),
 (('2024-01', 'Books'), 2),
 (('2024-01', 'Beauty Products'), 1),
 (('2024-01', 'Sports'), 3),
 (('2024-01', 'Electronics'), 1),
 (('2024-01', 'Home Appliances'), 1),
 (('2024-01', 'Clothing'), 2),
 (('2024-01', 'Books'), 3),
 (('2024-01', 'Beauty Products'), 1),
 (('2024-01', 'Sports'), 2),
 (('2024-01', 'Electronics'), 2),
 (('

In [9]:
# Ap dung reduceByKey de dem tong so luong tung loai trong moi thang

result_rdd = pair_rdd.reduceByKey(lambda x,y: x+y)

In [10]:
result_rdd.collect()

[(('2024-01', 'Electronics'), 10),
 (('2024-01', 'Books'), 11),
 (('2024-02', 'Home Appliances'), 8),
 (('2024-02', 'Clothing'), 19),
 (('2024-02', 'Beauty Products'), 7),
 (('2024-02', 'Sports'), 15),
 (('2024-03', 'Electronics'), 10),
 (('2024-03', 'Books'), 17),
 (('2024-04', 'Books'), 15),
 (('2024-04', 'Electronics'), 9),
 (('2024-05', 'Home Appliances'), 10),
 (('2024-05', 'Clothing'), 17),
 (('2024-05', 'Beauty Products'), 5),
 (('2024-05', 'Sports'), 10),
 (('2024-06', 'Clothing'), 19),
 (('2024-06', 'Beauty Products'), 6),
 (('2024-06', 'Sports'), 8),
 (('2024-06', 'Home Appliances'), 7),
 (('2024-07', 'Clothing'), 18),
 (('2024-07', 'Beauty Products'), 5),
 (('2024-07', 'Sports'), 5),
 (('2024-07', 'Home Appliances'), 7),
 (('2024-08', 'Beauty Products'), 6),
 (('2024-08', 'Sports'), 9),
 (('2024-08', 'Home Appliances'), 5),
 (('2024-08', 'Clothing'), 12),
 (('2024-01', 'Home Appliances'), 6),
 (('2024-01', 'Clothing'), 16),
 (('2024-01', 'Beauty Products'), 5),
 (('2024-01',

In [11]:
# Tìm sản phẩm bán chạy nhất trong mỗi tháng
best_seller = result_rdd.map(lambda x: (x[0][0], (x[0][1], x[1]))) \
                           .groupByKey() \
                           .mapValues(lambda x: max(x, key=lambda y: y[1]))

In [12]:
best_seller.collect()

[('2024-01', ('Sports', 17)),
 ('2024-03', ('Clothing', 24)),
 ('2024-04', ('Clothing', 20)),
 ('2024-02', ('Clothing', 19)),
 ('2024-05', ('Clothing', 17)),
 ('2024-06', ('Clothing', 19)),
 ('2024-07', ('Clothing', 18)),
 ('2024-08', ('Books', 13))]

Như kết quả trên, dễ dàng nhận thấy tháng 1 mặt hàng Sport bán chạy nhất, tiếp theo từ tháng 2 đến \
tháng 7 mặt hàng Clothing bán chạy nhất. Tháng 8 thì mặt hàng sách bán chạy nhất

*Ở trên dùng spark rdd, trong các bài sau, ta sẽ dùng **spark DataFrame** hay **spark SQL** sẽ sử lí dữ liệu thời gian dễ hơn*

**Bài 2: Tính trung bình điểm của học sinh theo từng lớp từ một RDD chứa dữ liệu điểm số.**

Ta sẽ sử dụng dataset student_performance_data.csv để làm. Vì đặc thù dữ liệu, ta sẽ tính điểm trung bình theo thang điểm 4 và tính theo từng chủng tộc, xem điểm số trung bình của chủng tộc nào cao nhất.

In [13]:
# Load du lieu
student_rdd = sc.textFile("material_1/student_performance_data.csv")

In [15]:
student_rdd.take(5)

['StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass',
 '1001,17,1,0,2,19.833722807854713,7,1,2,0,0,1,0,2.929195591667681,2.0',
 '1002,18,0,0,1,15.40875605584674,0,0,1,0,0,0,0,3.042914833436377,1.0',
 '1003,15,0,2,3,4.21056976881226,26,0,2,0,0,0,0,0.1126022544661815,4.0',
 '1004,17,1,0,3,10.028829473958215,14,0,3,1,0,0,0,2.0542181397029484,3.0']

In [16]:
# Bo dong dau tien di
firstRow = student_rdd.first()

student_rdd_no_header = student_rdd.filter(lambda x: x != firstRow)

student_rdd_no_header.take(5)

['1001,17,1,0,2,19.833722807854713,7,1,2,0,0,1,0,2.929195591667681,2.0',
 '1002,18,0,0,1,15.40875605584674,0,0,1,0,0,0,0,3.042914833436377,1.0',
 '1003,15,0,2,3,4.21056976881226,26,0,2,0,0,0,0,0.1126022544661815,4.0',
 '1004,17,1,0,3,10.028829473958215,14,0,3,1,0,0,0,2.0542181397029484,3.0',
 '1005,17,1,0,2,4.6724952729713305,17,1,3,0,0,0,0,1.2880611817953875,4.0']

Chú thích:
- Ethnicity
    - 0: Caucasian
    - 1: African American
    - 2: Asian
    - 3: Other
- ParentalSupport
    - 0: None
    - 1: Low
    - 2: Moderate
    - 3: High
    - 4: Very High

In [21]:
# Tao pair rdd chua du lieu (Ethnicity, ParentalSupport)

def parse_line(line):
    fields = line.split(",")
    # Kiểm tra xem dòng có đủ cột và dữ liệu có hợp lệ không
    if len(fields) == 15:  # Số cột hợp lệ trong file
        try:
            ethnicity = fields[3]
            parentalSupport = int(fields[8])
            return (ethnicity, parentalSupport)
        except ValueError:
            return None  # Trả về None nếu không thể chuyển đổi số lượng
    else:
        return None  # Trả về None nếu thiếu trường
    

pair_rdd = student_rdd_no_header.map(parse_line).filter(lambda x: x is not None)

pair_rdd.take(5)

[('0', 2), ('0', 1), ('2', 2), ('0', 3), ('0', 3)]

In [22]:
average_rdd = pair_rdd.groupByKey().mapValues(lambda x : sum(x)/len(x))

In [23]:
average_rdd.collect()

[('0', 2.12013256006628),
 ('1', 2.050709939148073),
 ('2', 2.1638297872340426),
 ('3', 2.2027027027027026)]

Thấy tỉ lệ điểm củng ngang ngang nhau, chứng tỏ học vấn không phụ thuộc vào chủng tộc :D

**Bài 3: Thực hiện phép "cogroup" để kết hợp dữ liệu từ hai RDD có cùng khóa nhưng giá trị khác nhau.**

In [24]:
# sales_rdd1: Mã đơn hàng, Sản phẩm và Số lượng
sales_rdd1 = sc.parallelize([
    ('10001', ('iPhone 14 Pro', 2)),
    ('10002', ('Dyson V11 Vacuum', 1)),
    ('10003', ('Levi\'s 501 Jeans', 3)),
    ('10004', ('The Da Vinci Code', 4))
])

# sales_rdd2: Mã đơn hàng, Giá mỗi sản phẩm và Tổng giá
sales_rdd2 = sc.parallelize([
    ('10001', (999.99, 1999.98)),
    ('10002', (499.99, 499.99)),
    ('10003', (69.99, 209.97)),
    ('10005', (89.99, 89.99))
])



In [25]:
# Thực hiện phép cogroup để kết hợp dữ liệu từ cả hai RDD theo mã đơn hàng
cogrouped_rdd = sales_rdd1.cogroup(sales_rdd2)

# Lấy kết quả để xem
cogrouped_result = cogrouped_rdd.collect()



In [26]:
for order_id, (info1, info2) in cogrouped_result:
    print(f"Mã đơn hàng: {order_id}")
    print(f"Thông tin từ sales_rdd1: {list(info1)}")
    print(f"Thông tin từ sales_rdd2: {list(info2)}")
    print("------")

Mã đơn hàng: 10003
Thông tin từ sales_rdd1: [("Levi's 501 Jeans", 3)]
Thông tin từ sales_rdd2: [(69.99, 209.97)]
------
Mã đơn hàng: 10002
Thông tin từ sales_rdd1: [('Dyson V11 Vacuum', 1)]
Thông tin từ sales_rdd2: [(499.99, 499.99)]
------
Mã đơn hàng: 10001
Thông tin từ sales_rdd1: [('iPhone 14 Pro', 2)]
Thông tin từ sales_rdd2: [(999.99, 1999.98)]
------
Mã đơn hàng: 10005
Thông tin từ sales_rdd1: []
Thông tin từ sales_rdd2: [(89.99, 89.99)]
------
Mã đơn hàng: 10004
Thông tin từ sales_rdd1: [('The Da Vinci Code', 4)]
Thông tin từ sales_rdd2: []
------


In [27]:
spark.stop()